<a href="https://colab.research.google.com/github/viniciusrpb/cic0269_natural_language_processing/blob/main/lectures/aula_named_entity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicação: Reconhecimento de Entidades Nomeadas

https://www.tensorflow.org/datasets/catalog/conll2003


CoNLL 2003
This dataset includes 1,393 English and 909 German news articles. The English-language corpus is free, but the German corpus comes at $75, unfortunately. This is the only corpus that costs something in this post. To build the English-language corpus you need the RCV1 Reuters corpus. You will obtain access a couple days after submitting the organisational and individual agreement at no charge.

Entities are annotated with LOC (location), ORG (organisation), PER (person) and MISC (miscellaneous). 

In [ ]:
!pip install -U tensorflow-datasets
!pip install keras
!pip install tensorflow
!pip install keras-crf

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import pandas as pd
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,Dropout,LSTM,Bidirectional,TimeDistributed
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report
from collections import Counter
from keras_crf import CRFModel
import tensorflow_addons as tfa

In [3]:
def preprocessDataFrame(df):

    dic = {}
    dic['tokens'] = []

    for text in df['tokens']:
        tokens = []
        for x in text:
            tokens.append(x.decode('utf-8'))
        l = " ".join(tokens)
        dic['tokens'].append(l.split())
        
    res_df = pd.DataFrame.from_dict(dic)
    res_df['ner'] = df['ner']
    return res_df

In [4]:
ds_train = tfds.load('conll2003', split='train', shuffle_files=True)
ds_valid = tfds.load('conll2003', split='dev', shuffle_files=False)
ds_test = tfds.load('conll2003', split='test', shuffle_files=False)

In [5]:
df_train = preprocessDataFrame(tfds.as_dataframe(ds_train))
df_valid = preprocessDataFrame(tfds.as_dataframe(ds_valid))
df_test = preprocessDataFrame(tfds.as_dataframe(ds_test))

In [6]:
df_train

,tokens,ner
0,"["", If, they, 're, saying, at, least, 20, perc...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[Lauck, 's, lawyer, vowed, he, would, appeal, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Thailand, 's, powerful, military, thinks, the...","[5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, ..."
3,"[A, forensic, scientist, who, examined, the, s...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,"[Werder, Bremen, 3, 0, 1, 2, 4, 6, 1]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
14037,"["", He, was, not, involved, ...]","[0, 0, 0, 0, 0, 0]"
14038,"["", It, goes, without, saying, that, we, 're, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
14039,"[Bowling, :]","[0, 0]"
14040,"[National, League]","[7, 8]"


In [7]:
def label2int():
    iob_labels = ["B", "I"]
    ner_labels = ["PER", "ORG", "LOC", "MISC"]
    all_labels = [(label1, label2) for label2 in ner_labels for label1 in iob_labels]
    all_labels = ["-".join([a, b]) for a, b in all_labels]
    dic = dict(zip(range(1, len(all_labels) + 1), all_labels))
    dic[0] = 'O'
    return dic 

In [8]:
int2tag = label2int()

int2tag

{1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC',
 0: 'O'}

In [9]:
tag2int = {}
for key in int2tag:
    value = int2tag[key]
    tag2int[value] = key
print(tag2int)

num_labels = len(tag2int)

{'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8, 'O': 0}


Cria o vocabulário

In [10]:
word2index = {}
word2index['<OOV>'] = 0
word2index['<BEG>'] = 1
word2index['<END>'] = 2

ind = 3
for text in df_train['tokens']:
    for word in text:
        if word not in word2index:
            word2index[word] = ind
            ind+=1

index2word = {}
for key in word2index:
    value = word2index[key]
    index2word[value] = key

In [11]:
def rebuild_datasets(df,labels):

    data = []
    novel_labels = []

    idx = 0
    for text in df:

        data.append(['<BEG>'])
        novel_labels.append(0)

        for i in range(0,len(text)):
            lista = ['<BEG>']+text[:i+1]
            data.append(lista)
            if i < len(text)-1:
                novel_labels.append(labels[idx][i])
            else:
                novel_labels.append(0)
        idx+=1

        data.append(lista+['<END>'])
        novel_labels.append(0)

    return data,novel_labels

In [12]:
x_train,y_train_int = rebuild_datasets(df_train['tokens'],df_train['ner'])
x_valid,y_valid_int = rebuild_datasets(df_valid['tokens'],df_valid['ner'])
x_test,y_test_int = rebuild_datasets(df_test['tokens'],df_test['ner'])

Verificando se está tudo certo

In [13]:
for i,word in enumerate(x_train[33]):
    print(f'Word: {word} text label: {int2tag[y_train_int[i]]} int label: {y_train_int[i]} ')

Word: <BEG> text label: O int label: 0 
Word: " text label: O int label: 0 
Word: If text label: O int label: 0 
Word: they text label: O int label: 0 
Word: 're text label: O int label: 0 
Word: saying text label: O int label: 0 
Word: at text label: O int label: 0 
Word: least text label: O int label: 0 
Word: 20 text label: O int label: 0 
Word: percent text label: O int label: 0 
Word: , text label: O int label: 0 
Word: then text label: O int label: 0 
Word: their text label: O int label: 0 
Word: internal text label: O int label: 0 
Word: forecasts text label: O int label: 0 
Word: are text label: O int label: 0 
Word: probably text label: O int label: 0 
Word: saying text label: O int label: 0 
Word: 25 text label: O int label: 0 
Word: or text label: O int label: 0 
Word: 30 text label: O int label: 0 
Word: percent text label: O int label: 0 
Word: , text label: O int label: 0 
Word: " text label: O int label: 0 
Word: said text label: O int label: 0 
Word: one text label: O i

In [14]:
max_length = max([len(sentence) for sentence in x_train])
max_length

115

In [15]:
def text2sequences(data,vocab):
    corpus_int = []
    for instance in data:
        instance_int = []
        for term in instance:
            if term in vocab:
                instance_int.append(vocab[term])
            else:
                instance_int.append(vocab['<OOV>'])
        corpus_int.append(instance_int)
    
    return corpus_int

In [16]:
train_sequences = text2sequences(x_train,word2index)
valid_sequences = text2sequences(x_valid,word2index)
test_sequences = text2sequences(x_test,word2index)

In [17]:
trunc_type = 'post'
padding_type = 'pre'

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [18]:
y_train = to_categorical(y_train_int)
y_valid = to_categorical(y_valid_int)
y_test = to_categorical(y_test_int)

In [19]:
vocab_size = len(word2index)

In [20]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size+3,output_dim=64,input_length=max_length))
model.add(Bidirectional(LSTM(256,activation='relu')))
model.add(Dropout(0.5))
model.add(Dense(num_labels,activation='softmax'))

In [21]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 115, 64)           1512256   
                                                                 
 bidirectional (Bidirectiona  (None, 512)              657408    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 9)                 4617      
                                                                 
Total params: 2,174,281
Trainable params: 2,174,281
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
f1scoretfa = tfa.metrics.F1Score(num_classes=num_labels, average='macro',threshold=0.5)

sgd = SGD(learning_rate = 0.00005)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy',f1scoretfa])
history_fine = model.fit(train_padded,y_train,batch_size=32,validation_data=(valid_padded,y_valid),epochs=10)

Epoch 1/10
7241/7241 [==============================] - 3417s 471ms/step - loss: 2.0125 - accuracy: 0.8354 - f1_score: 0.0000e+00 - val_loss: 1.8339 - val_accuracy: 0.8538 - val_f1_score: 0.0000e+00
Epoch 2/10
7241/7241 [==============================] - 3334s 460ms/step - loss: 1.6453 - accuracy: 0.8562 - f1_score: 0.0000e+00 - val_loss: 1.4551 - val_accuracy: 0.8538 - val_f1_score: 0.0000e+00
Epoch 3/10
7241/7241 [==============================] - 3424s 473ms/step - loss: 1.2567 - accuracy: 0.8562 - f1_score: 7.4978e-04 - val_loss: 1.0641 - val_accuracy: 0.8538 - val_f1_score: 0.0000e+00
Epoch 4/10
3329/7241 [============>.................] - ETA: 30:14 - loss: 0.9796 - accuracy: 0.8571 - f1_score: 0.0742

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['f1_score'])
plt.plot(history_fine.history['val_f1_score'])
plt.title('model f1 score')
plt.ylabel('f1score')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

Teste

In [ ]:
y_prob = model.predict(test_padded)
y_pred = np.argmax(y_prob,axis=1)

In [ ]:
print(classification_report(y_test_int,y_pred))

Deployment

In [ ]:
sentenca_teste = "Mayara is living in Spain which its land urges for coffee"
sentenca_int = text2sequences(sentenca_teste,word2index)
sentenca_padded = pad_sequences(sentenca_int, maxlen=max_length, padding=padding_type, truncating=trunc_type)

y_prob = model.predict(sentenca_padded)
y_pred = np.argmax(y_prob,axis=1)

for idx,word in enumerate(sentenca_teste):
    print(f'{word} is label {int2tag[y_pred[idx]]}')